# ■ LangChain

LLMのラッパー

- [LangChain公式ドキュメント](https://python.langchain.com/docs/get_started/introduction)
- [LangChain | GitHub](https://github.com/langchain-ai/langchain)
- [Get started | LangChain](https://python.langchain.com/docs/expression_language/get_started)

## # モジュール

- [Modules | LangChain](https://python.langchain.com/docs/modules/)
  - [Model I/O](https://python.langchain.com/docs/modules/model_io/)
  - [Prompts](https://python.langchain.com/docs/modules/model_io/prompts/)
  - [Chains](https://python.langchain.com/docs/modules/chains)
  - [Indexing](https://python.langchain.com/docs/modules/data_connection/indexing)
  - [Memory](https://python.langchain.com/docs/modules/memory/)
  - [Agents](https://python.langchain.com/docs/modules/agents/)

# ■ Models モジュール

LangChainで使用する機械学習のモデルのこと。以下の3種類がある。

- LLMs  
OpenAIのCompletions API(gpt-3.5-turbo-instructなど)の大規模言語モデル
- Chat Models  
OpenAIのChat API(gpt-4, gpt-3.5-turboなど)の大規模言語モデル
- Text Embedding Models
テキストをベクトル化するモデル。

In [1]:
import os
openai_api_key = os.environ["OPENAI_API_KEY"]

## LLMs

- [Modules - Model I/O - LLMS | LangChain公式](https://python.langchain.com/docs/modules/model_io/llms/)

内部的にはCompletionsAPIが利用される

In [4]:
# LLMs
from langchain.llms import OpenAI
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)
result = llm.predict("自己紹介してください")
print(result)



私は、山田太郎と申します。東京都出身で、現在は大学生として都内の大学に通っています。趣味はスポーツ観戦や音楽鑑賞で、特にサッカーやロックバンドが好きです。将来の夢は、国際的な企業で働くことで、留学経験も積んで自分を磨きたいと思っています。また、人とのコミュニケーションを大切にし、常に新しいことに挑戦することで成長していきたいと考えています。よろしくお願いします。


## Chat Models

- [Modules - Model I/O - Chat Models | LangChain公式](https://python.langchain.com/docs/modules/model_io/chat/)

内部的には ChatAPIが利用される

In [5]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0)
result = llm.predict("自己紹介してください")
print(result)

はじめまして、私はAIアシスタントです。自然言語処理技術を用いて、様々な質問や会話に対応することができます。お手伝いが必要なことがあれば、遠慮なくお知らせください。どうぞよろしくお願いいたします。


# ■ Prompts モジュール

- [Modules - Model I/O - Prompts | LangChain公式](https://python.langchain.com/docs/modules/model_io/prompts/)

モデルへの入力を組み立てるモジュール。以下の4つの要素がある。

- Prompt Templetes  
プロンプトをテンプレート化できる
- Chat Prompt Templates
- Example Selectors
- Output Parsers

## Prompt Templates

プロンプトをテンプレート化することができる。  
あくまで文字列を編集するだけでAPIをコールするわけではない。

In [6]:
from langchain.prompts import PromptTemplate

template = """
次のコマンドの概要を説明してください。

コマンド: {command}
"""

prompt = PromptTemplate(
  template=template,
  input_variables=["command"],
)
result = prompt.format(command="ls")
print(result)


次のコマンドの概要を説明してください。

コマンド: ls



# ■ Chains モジュール

- [Modules - Chains | LangChain公式](https://python.langchain.com/docs/modules/chains)

chainsは、モジュール(Models, Templates, Chainsなど)を連結する。

In [8]:
import langchain
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

langchain.verbose = True
chat = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0)

template = """
次のコマンドの概要を説明してください。

コマンド: {command}
"""

prompt = PromptTemplate(
  template=template,
  input_variables=["command"],
)

chain = LLMChain(llm=chat, prompt=prompt)
result = chain.run("ls")
print(result)



> Entering new LLMChain chain...
Prompt after formatting:

次のコマンドの概要を説明してください。

コマンド: ls


> Finished chain.
lsコマンドは、リスト（list）の略で、指定されたディレクトリ内のファイルやディレクトリの一覧を表示するためのコマンドです。デフォルトではカレントディレクトリの内容を表示しますが、任意のディレクトリを指定することもできます。lsコマンドを実行することで、ファイルやディレクトリの名前や属性、更新日時などの情報を確認することができます。


## SimpleSequentialChain

ChainとChainを直列に連結できる。

<img src="./img/simple_sequential_chain.png">

In [9]:
# 「ステップバイステップで考える(COT)」 と 「要約」 の組み合わせ

from langchain.chains import SimpleSequentialChain

langchain.verbose = True

# 「ステップバイステップで考える(COT)」 Prompt と Chain を用意
chat = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0)

cot_template = """
以下の質問に回答してください。

### 質問 ###
{question}
### 質問終了 ###

ステップバイステップで考えましょう。
"""

cot_prompt = PromptTemplate(
  template=cot_template,
  input_variables=["question"],
)

cot_chain = LLMChain(llm=chat, prompt=cot_prompt)

# 「要約」 Prompt と Chain を用意
summarize_template = """
入力を結論だけ一言に要約してください。

### 入力 ###
{input}
### 入力終了 ###
"""

summarize_prompt = PromptTemplate(
  input_variables=["input"],
  template=summarize_template,
)

summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)


# 2 つの Chain を直列に繋ぐ
cot_summarize_chain = SimpleSequentialChain(
  chains=[cot_chain, summarize_chain]
)

# 実行
result = cot_summarize_chain(
  "私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？"
)

print(result["output"])



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

以下の質問に回答してください。

### 質問 ###
私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？
### 質問終了 ###

ステップバイステップで考えましょう。


> Finished chain.
1. 最初に市場で10個のリンゴを買いました。
2. 隣人に2つ、修理工に2つ渡しました。残りは10 - 2 - 2 = 6個です。
3. その後、5つのリンゴを追加で購入しました。残りは6 + 5 = 11個です。
4. 最後に1つのリンゴを食べました。残りは11 - 1 = 10個です。

したがって、最終的には10個のリンゴが残ります。


> Entering new LLMChain chain...
Prompt after formatting:

入力を結論だけ一言に要約してください。

### 入力 ###
1. 最初に市場で10個のリンゴを買いました。
2. 隣人に2つ、修理工に2つ渡しました。残りは10 - 2 - 2 = 6個です。
3. その後、5つのリンゴを追加で購入しました。残りは6 + 5 = 11個です。
4. 最後に1つのリンゴを食べました。残りは11 - 1 = 10個です。

したがって、最終的には10個のリンゴが残ります。
### 入力終了 ###


> Finished chain.
最終的には10個のリンゴが残ります。

> Finished chain.
最終的には10個のリンゴが残ります。


# ■ Output Parsers

- [Modules - Model I/O - Output Parsers | LangChain](https://python.langchain.com/docs/modules/model_io/output_parsers/)

LLMの応答を抽出して、Pythonオブジェクトにマッピングするといった機能を提供する。  
他にもOutput Parserには、不正な形式でうまく解析できなかった場合に LLM に形式を整えてもらうような機能がある。  

Promptsには他にも、Few-shotプロンプティングの例を埋め込むための「ExampleSelectors」という機能もある

In [12]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, validator
from typing import List

langchain.verbose = True

class Recipe(BaseModel):
  ingredients: List[str] = Field(description="ingredients of the dish")
  step: List[str] = Field(description="steps to cook the dish")

template = """料理のレシピを教えてください。

{format_instructions}

料理名: {dish}
"""

# 出力値のフォーマットを指示するためのプロンプトを生成する
parser = PydanticOutputParser(pydantic_object=Recipe)

prompt = PromptTemplate(
  template=template,
  input_variables=["dish"],
  partial_variables={
    "format_instructions": parser.get_format_instructions()
  },
)

# Chain を実行
chat = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0)
chain = LLMChain(llm=chat, prompt=prompt)
output = chain.run(dish="カレー")
print("=== === === output === === ===")
print(output)

# 出力をPydanticオブジェクトにパース
recipe = parser.parse(output)
print("=== === === recipe === === ===")
print(recipe)



> Entering new LLMChain chain...
Prompt after formatting:
料理のレシピを教えてください。

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "step": {"description": "steps to cook the dish", "items": {"type": "string"}, "title": "Step", "type": "array"}}, "required": ["ingredients", "step"]}
```

料理名: カレー


> Finished chain.
=== === === output === === ===
```json
{
    "ingredients": [
        "玉ねぎ",
        "人参",
        "じゃがいも",
        "牛肉",
        "カレールー",
  